In [2]:
import pandas as pd
import os
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [5]:
def scrap_app_category(app_name):
    url = f'https://play.google.com/store/apps/details?id={app_name}'
    response = requests.get(url)
    if response.status_code == 200:
        # parsujemy zawartość strony
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # znajdujemy konkretny <a> na stronie za pomocą jego klasy
        target_anchor = soup.find('a', class_='WpHeLc VfPpkd-mRLv6 VfPpkd-RLmnJb')
        
        # sprawdzamy, czy znaleziono <a> o podanej klasie
        if target_anchor:
            #print(target_div.prettify())
            return str(target_anchor.attrs['aria-label'])
            
        else:
            return None
    else:
       return None

In [ ]:
package_list = []
for iter in range(1, 8):
    ###########  W MIEJSCU /.../ PODAJ ŚCIEŻKĘ DO DANYCH PO PRZETWORZENIU ###########
    iter_path = f"C:/.../aggregated_data/i_0{iter}/"
    for i, user_name in tqdm(enumerate(os.listdir(iter_path))):
        user_path = iter_path+user_name

        userstats = pd.read_csv(user_path+'usageStats.csv')
        package_list.extend(list(userstats['packageName']))
        package_list = set(package_list)

In [ ]:
print("Liczba unikalnych aplikacji dla wszystkich uczestników wynosi:", len(package_list))

In [ ]:
# scrapowanie danych
app_category_df = pd.DataFrame()
for app_name in package_list:
    category = scrap_app_category(app_name)
    app_category_df = app_category_df.append({'packageName': app_name, 'category': category}, ignore_index=True)

In [8]:
print(app_category_df['category'].unique())

['Tools' 'Lifestyle' 'Casual' 'Books & Reference' 'Travel & Local'
 'System' 'Productivity' 'Health & Fitness' 'Food & Drink' 'Education'
 'Puzzle' 'Music & Audio' 'Medical' 'Business' 'Weather' 'Strategy'
 'Sports' 'Shopping' 'Finance' 'Simulation' 'Entertainment'
 'News & Magazines' 'Board' 'Communication' 'Photography' 'Parenting'
 'Role Playing' 'Video Players & Editors' 'Maps & Navigation' 'Racing'
 'Beauty' 'Social' 'Personalization' 'Action' 'Art & Design'
 'Auto & Vehicles' 'Trivia' 'Word' 'Dating' 'Events' 'Adventure' 'Card'
 'Casino' 'House & Home' 'Comics']


In [12]:
# agregowanie kategorii do ogólnych grup
category_to_agg_category = {'Games':['Puzzle', 'Strategy', 'Board', 'Action', 'Racing', 'Casual', 'Simulation', 'Role Playing', 'Adventure', 'Card', 'Trivia', 'Word', 'Casino'],
                            'Communication':['Communication'],
                            'Reading':['Books & Reference', 'News & Magazines', 'Comics'],
                            'Health':['Health & Fitness', 'Medical'],
                            'Music & Audio':['Music & Audio'],
                            'Video':['Video Players & Editors', 'Entertainment'],
                            'Social':['Social', 'Dating'],
                            'Travel':['Travel & Local', 'Maps & Navigation',],
                            'Others':['Tools', 'System', 'Lifestyle', 'Food & Drink', 'Business', 'Photography',
                                      'Productivity', 'Education', 'Weather', 'Sports',
                                      'Finance', 'Parenting', 'Beauty', 'Personalization', 'Art & Design',
                                      'Auto & Vehicles', 'Events', 'House & Home', 'Shopping'],
                            }
category_to_agg_category = {cat: agg_cat for agg_cat, cat in category_to_agg_category.items() for cat in cat}
app_category_df['agg_category'] = app_category_df['category'].map(category_to_agg_category)

In [17]:
# zapisywanie danych do pliku
###########  UWAGA - PRZY ZAPISYWANIU ORYGINALNY PLIK ZOSTANIE NADPISANY (KATEGORIE DODANE RĘCZNIE ZOSTANĄ USUNIĘTE) ###########
#app_category_df.to_csv("C:/.../dataset/apps_category.csv", index=False)